In [1]:
import os
import cv2
from PIL import Image, ImageDraw, ImageFont
import matplotlib.pyplot as plt
import numpy as np

from tqdm import tqdm

In [2]:
from paddleocr import PaddleOCR
ocr = PaddleOCR(use_angle_cls = True, use_gpu= False, show_log=False)

In [3]:
labels_path = 'data/labels/'
labels_all = os.listdir(labels_path)

img_dir = 'data/images_test/'
write_dir = './results/'

for txt_label in tqdm(labels_all):
    txt_ = labels_path + txt_label
    # print(txt_)
    # break
    # 打开图片
    img_open = img_dir + txt_label[:-4] + '.jpg'
    img = cv2.imread(img_open)
    height, width, _ = img.shape
    # 打开txt文件, 并将归一化的坐标转换回去
    with open(txt_, 'r', encoding='utf-8') as f:
        lines = f.readlines()
        # print(lines)
        for line in lines:
            # print(line)
            id = int(line.split(' ')[0])
            x = float(line.split(' ')[1])
            y = float(line.split(' ')[2])
            w = float(line.split(' ')[3])
            h = float(line.split(' ')[4])
            
            # x * width * 2 = x1 + x3
            # w * width     = x3 - x1
            x1 = round((x * width * 2 - w * width) / 2, 2)
            x3 = round((x * width * 2 + w * width) / 2, 2)
            # y * height * 2 = y1 + y3
            # h * height     = y3 - y1
            y1 = round((y * height * 2 - h * height) / 2, 2)
            y3 = round((y * height * 2 + h * height) / 2, 2)
            
            x2 = x1
            y2 = y3
            x4 = x3
            y4 = y1
            # print(x1, y1, x3, y3)            
            # 裁剪、识别         
            crop = img[int(y1):int(y3), int(x1):int(x3)]
            # cv2.imshow('img', crop)
            text = ocr.ocr(crop, cls=True)
            # print(text[0])
            if text[0] == []:
                res_write = str(x1) + ',' + str(y1) + ',' + str(x2) + ',' + str(y2) + ',' + str(x3) + ',' + str(y3) + ',' + str(x4) + ',' + str(y4) + ',' + '###' + "\n"
            else:
                res_write = str(x1) + ',' + str(y1) + ',' + str(x2) + ',' + str(y2) + ',' + str(x3) + ',' + str(y3) + ',' + str(x4) + ',' + str(y4) + ',' + text[0][0][1][0] + '\n'
            with open(write_dir + txt_label, 'a', encoding='utf-8') as resf:
                resf.write(res_write)
            resf.close()
    f.close()
# cv2.waitKey(0)
# cv2.destroyAllWindows()

100%|██████████| 10000/10000 [5:00:38<00:00,  1.80s/it] 
